In [9]:
VER = 1

In [10]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [11]:
import pandas as pd, numpy as np
import pickle, glob, gc

from collections import Counter
import itertools

# multiprocessing 
import psutil
N_CORES = psutil.cpu_count()     # Available CPU cores
print(f"N Cores : {N_CORES}")
from multiprocessing import Pool

N Cores : 2


In [12]:
type_labels = {'clicks':0, 'carts':1, 'orders':2}

def load_test(files):    
    dfs = []
    for e, chunk_file in enumerate(glob.glob(files)):
        chunk = pd.read_parquet(chunk_file)
        chunk.ts = (chunk.ts/1000).astype('int32')
        chunk['type'] = chunk['type'].map(type_labels).astype('int8')
        dfs.append(chunk)
    return pd.concat(dfs).reset_index(drop=True) #.astype({"ts": "datetime64[ms]"})

In [19]:
train = load_test('/content/drive/MyDrive/Colab Notebooks/kaggle/OTTO/dataset/otto-validation/test_parquet/*')
print('train data has shape',train.shape)

train data has shape (7683577, 4)


In [20]:
def user_process(df):
    df = df.groupby('session').agg({'session':'count','aid':'nunique','type':'mean'})
    df.columns = ['user_user_count','user_item_count','user_buy_ratio']
    return df

In [21]:
train = user_process(train)
train.to_parquet(f'/content/drive/MyDrive/Colab Notebooks/kaggle/OTTO/dataset/candidate/user/val_user{VER}.pqt')

In [22]:
train.head()

,user_user_count,user_item_count,user_buy_ratio
session,,,
11098528,1,1,0.000000
11098529,1,1,0.000000
11098530,6,2,0.166667
11098531,24,11,0.333333
11098532,2,2,0.000000


In [23]:
del train
_ = gc.collect()

In [26]:
test = load_test('/content/drive/MyDrive/Colab Notebooks/kaggle/OTTO/dataset/original/test_parquet/*')
print('Test data has shape',test.shape)

Test data has shape (6928123, 4)


In [27]:
test = user_process(test)
test.to_parquet(f'/content/drive/MyDrive/Colab Notebooks/kaggle/OTTO/dataset/candidate/user/test_user{VER}.pqt')